In [1]:
import itertools
import numpy as np
import qiskit.quantum_info as qi
from qiskit.visualization import array_to_latex
from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn
from kaleidoscope import bloch_sphere

class Basis:
    def __init__(self, data, name=''):
        self.data = data
        self.index = 0
        self.num = len(data)
        self.num_qubits = data[0].num_qubits
        self.dim = 2**self.num_qubits
        self.name = name
        self.pseudoinv = self._pseudoinverse()
        
    def __iter__(self):
        self.index = 0
        return self
        
    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration
        result = self.data[self.index]
        self.index += 1
        return result
        
    def _pseudoinverse(self):
        matrix = []
        for mat in self.data:
            temp = mat.conjugate().data.reshape(-1,1)
            temp = [element[0] for element in temp]
            matrix.append(np.array(temp))
        matrix = np.array(matrix)
        pseudoinv = np.matmul(
            np.linalg.inv(
                np.matmul(
                    matrix.transpose(), 
                    matrix)
            ), 
            matrix.transpose()
        )
        return pseudoinv

class QSTdata:
    def __init__(self, 
                 runs,
                 fidelity,
                 err_lo=None,
                 err_hi=None,
                 err_min=None,
                 err_max=None):
        self.runs = runs
        self.fidelity = fidelity
        self.err_lo = err_lo
        self.err_hi = err_hi
        self.err_min = err_min
        self.err_max = err_max
        
sicpovm_1qb = Basis(
    [
    qi.Statevector([1,0]).to_operator()/2,
    qi.Statevector([1/np.sqrt(3),
                    np.sqrt(2/3)]).to_operator()/2,
    qi.Statevector([1/np.sqrt(3),
                    np.sqrt(2/3) * np.exp(1j*2*np.pi/3)]
                  ).to_operator()/2,
    qi.Statevector([1/np.sqrt(3),
                    np.sqrt(2/3) * np.exp(1j*4*np.pi/3)]
                  ).to_operator()/2
],
    name='SIC-POVM')

def generate_sicpovm_2qb():
    x = np.sqrt(2 + np.sqrt(5))
    norm = 4*(5 + np.sqrt(5))
    basis = [
        qi.Statevector([x,1,1,1]).to_operator()/norm,
        qi.Statevector([x,1,-1,-1]).to_operator()/norm,
        qi.Statevector([x,-1,1,-1]).to_operator()/norm,
        qi.Statevector([x,-1,-1,1]).to_operator()/norm,
        qi.Statevector([1j,x,1,-1j]).to_operator()/norm,
        qi.Statevector([1j,x,-1,1j]).to_operator()/norm,
        qi.Statevector([-1j,x,1,1j]).to_operator()/norm,
        qi.Statevector([-1j,x,-1,-1j]).to_operator()/norm,
        qi.Statevector([1j,1j,x,-1]).to_operator()/norm,
        qi.Statevector([1j,-1j,x,1]).to_operator()/norm,
        qi.Statevector([-1j,1j,x,1]).to_operator()/norm,
        qi.Statevector([-1j,-1j,x,-1]).to_operator()/norm,
        qi.Statevector([1j,1,-1j,x]).to_operator()/norm,
        qi.Statevector([1j,-1,1j,x]).to_operator()/norm,
        qi.Statevector([-1j,1,1j,x]).to_operator()/norm,
        qi.Statevector([-1j,-1,-1j,x]).to_operator()/norm
    ]
    return Basis(basis, name='SIC-POVM 2-Qubit')

sicpovm_2qb = generate_sicpovm_2qb()
sicpovm = {
    1: sicpovm_1qb,
    2: sicpovm_2qb
}

def generate_pauli_matrices(num):
    pauli = [
        qi.Operator([[1,0],[0,1]]),
        qi.Operator([[0,1],[1,0]]),
        qi.Operator([[0,-1j],[1j,0]]),
        qi.Operator([[1,0],[0,-1]])
    ]
    temp_pauli = pauli
    new_pauli = pauli
    for _ in range(num-1):
        new_pauli = []
        for operator1 in temp_pauli:
            for operator2 in pauli:
                new_pauli.append(operator1.tensor(operator2))
        temp_pauli = new_pauli
    return new_pauli
    
def generate_pauli_nqb(num):
    new_pauli = generate_pauli_matrices(num)
    basis = []
    dim = 2**num
    for operator in new_pauli[1:]:
        basis.append((new_pauli[0] + operator)/dim)
        basis.append((new_pauli[0] - operator)/dim)

    result = qi.Operator(np.zeros((dim, dim)))
    for operator in basis:
        result += operator
    trace = result.data.trace()
    for i, operator in enumerate(basis):
        basis[i] = operator / trace*dim
    return Basis(basis, name=f'{len(basis)}-Pauli')

pauli_1qb = generate_pauli_nqb(1)
pauli_2qb = generate_pauli_nqb(2)
pauli_povm = {
    1: pauli_1qb,
    2: pauli_2qb
}
    
def random_direction(num=4):
    n = []
    for _ in range(num):
        n.append(np.random.normal(0, 1, size=3))
        n[-1] /= np.sqrt(np.sum(n[-1]**2))
    return n
    
def atl(x, prefix='', source=False):
    if source:
        print(array_to_latex(x, prefix=prefix, source=True))
    else:
        display(array_to_latex(x, prefix=prefix))

def remove_negatives_and_diagonalize(arr):
    arr[arr < 0] = 0
    diagonal_matrix = np.diag(arr)
    return diagonal_matrix

def measure(probabilities, num=1):
    p_total = abs(1 - sum(probabilities))
    if p_total > 0.01:
        print(f'ERROR: Probabilities sum to 1 +/- {p_total}')

    counts = rng.multinomial(num, probabilities)
    estimators = counts / num
    return estimators

def qst(state, basis, 
        shots_max=400, 
        runs_per_shot=1, 
        step=1, 
        profile=True, 
        calc_errors=True):
    probabilities = []
    for projector in basis.data:
        probabilities.append(
            np.trace(
                np.matmul(
                    state.data, 
                    projector.data
                )
            ).real)
    if profile:
        shots = range(step*basis.num, (shots_max+1), step*basis.num)
    else:
        shots = [shots_max - (shots_max % basis.num)]
    fidelity_avg = []
    err_hi = []
    err_lo = []
    err_max = []
    err_min = []
    print('QST completion %', end=' ')
    for num_shots in shots:
        print(int(100*num_shots / shots_max), end=' ')
        fidelity = []
        for _ in range(runs_per_shot):
            estimator = measure(probabilities, num_shots/basis.num)
            state_reconstruct = qi.DensityMatrix(
                np.matmul(
                    basis.pseudoinv, 
                    estimator
                ).reshape(basis.dim, basis.dim))
            state_valid = make_psd_and_rescale_trace(state_reconstruct)
            fidelity.append(qi.state_fidelity(state_valid, state))
        fidelity = np.array(fidelity)
        avg = np.mean(fidelity)
        if profile:
            fidelity_avg.append(avg)
        else:
            fidelity_avg = avg
        if calc_errors:
            var_hi = fidelity[fidelity > avg]
            var_lo = fidelity[fidelity < avg]
            dev_hi = np.mean(var_hi)
            dev_lo = np.mean(var_lo)
            if profile:
                err_max.append(np.max(fidelity))
                err_min.append(np.min(fidelity))
                err_hi.append(dev_hi)
                err_lo.append(dev_lo)
            else:
                err_max = np.max(fidelity)
                err_min = np.min(fidelity)
                err_hi = dev_hi
                err_lo = dev_lo                
    if profile:
        fidelity_avg = np.array(fidelity_avg)
    print()
    if calc_errors:
        return QSTdata(shots, 
                       fidelity_avg, 
                       err_lo, 
                       err_hi, 
                       err_min, 
                       err_max)
    else:
        return QSTdata(shots, 
                       fidelity_avg)

def make_psd_and_rescale_trace(state):
    y = 0.5*(state.data + state.data.T.conjugate())
    eigvals, eigvecs = np.linalg.eig(y)
    eigvals = remove_negatives_and_diagonalize(eigvals)
    z = np.matmul(eigvecs, 
                  np.matmul(eigvals, 
                            eigvecs.T.conjugate()))
    z /= z.trace()
    return z

def generate_complex_numbers(num):
    angles = 2*np.pi*rng.uniform(size=num)
    z = np.exp(1j * angles)
    return z

def generate_real_complex_pair():
    x = rng.uniform()
    z = np.sqrt(1 - x**2) * generate_complex_numbers(1)
    pair = np.concatenate(([x], z))
    return pair

def generate_complex_pairs(num_pairs):
    z = generate_complex_numbers(2*num_pairs)
    norms = []
    for x in rng.uniform(size=num_pairs):
        norms.append(x)
        norms.append(np.sqrt(1 - x**2))
    return z * np.array(norms)

def normalize(x):
    norm = np.sqrt(np.sum(np.vdot(x, x)))
    return x / norm

def generate_n_qubits(num, entangled=False, density_matrix=False):
    if entangled:
        coefs = np.concatenate((generate_real_complex_pair(),
                                generate_complex_pairs(2**(num-1)-1)))
        coefs = normalize(coefs)
        state = qi.Statevector(coefs)
    else:
        state = qi.Statevector(generate_real_complex_pair())
        for _ in range(num-1):
            new_state = qi.Statevector(generate_real_complex_pair())
            state = state.tensor(new_state)

    if density_matrix:
        state = qi.DensityMatrix(state)
        
    return state

def generate_combinations(lst):
    combinations = []
    for r in range(1, len(lst)):
        combinations.extend([list(comb) for comb \
                             in itertools.combinations(lst, r)])
    return combinations

def list_subsystems(state):
    trace_out = generate_combinations(range(state.num_qubits))
    states_traced = []
    for subsys in trace_out:
        states_traced.append(qi.partial_trace(state, subsys))
    return states_traced, trace_out

def frobenius_norm(a, b):
    res = np.matmul(a.conj().T, b).trace()
    return res

def verify_povm(basis, print_res=False):
    valid = True
    result = np.zeros_like(basis.data[0])
    eigvals = []
    for operator in basis.data:
        result += operator.data
        eigvals.append(np.linalg.eigvals(operator.data))
        for val in eigvals[-1]:
            if abs(np.imag(val)) > 0.00000001:
                valid = False
            elif np.real(val) < -0.00000001:
                valid = False
    if not valid and print_res:
        print('Operators not PSD')
    if not np.allclose(result, np.identity(result.shape[0])):
        valid = False
        if print_res:
            print('Operators sum to')
            atl(result)
    if valid and print_res:
        print('Valid POVM')
    return valid

def generate_perturbed_sicpovm_1qb(amt=0.1):
    pauli = [
        np.array([[1,0],[0,1]], dtype=complex),
        np.array([[0,1],[1,0]], dtype=complex),
        np.array([[0,-1j],[1j,0]], dtype=complex),
        np.array([[1,0],[0,-1]], dtype=complex)
    ]
    n = random_direction()
    rotation = []
    for ni in n:
        rotation.append(pauli[0]*np.cos(amt*np.pi))
        for nj, sigma in zip(ni, pauli[1:]):
            rotation[-1] -= sigma*1j*np.sin(amt*np.pi)*nj
    basis_vecs = [
        qi.Statevector([1,0]),
        qi.Statevector([1/np.sqrt(3),
                        np.sqrt(2/3)]),
        qi.Statevector([1/np.sqrt(3),
                        np.sqrt(2/3) * np.exp(1j*2*np.pi/3)]),
        qi.Statevector([1/np.sqrt(3),
                        np.sqrt(2/3) * np.exp(1j*4*np.pi/3)])
    ]
    new_povm = []
    bloch_factor = 1
    trace = 0
    for rot, vec in zip(rotation, basis_vecs):
        vec = qi.Statevector(
            np.dot(rot, np.dot(vec.data, rot.conjugate().T))
        )
        bloch_factor = np.abs(vec[0]) / vec.data[0]
        vec *= bloch_factor
        new_povm.append(vec.to_operator())
        new_trace = new_povm[-1].data.trace()
        trace += new_trace
    for i, _ in enumerate(new_povm):
        new_povm[i] /= trace/2
    total = qi.Operator(np.zeros_like(new_povm[0]))
    for proj in new_povm:
        total += proj
    diff = total - np.identity(2)
    physical = True
    for i, _ in enumerate(new_povm):
        new_povm[i] -= diff/4
    new_povm = Basis(new_povm, name=f'SIC-POVM (p={amt})')
    physical = verify_povm(new_povm)
    return new_povm, physical

def commutator(operator1, operator2):
    res1 = np.matmul(operator1, operator2)
    res2 = np.matmul(operator2, operator1)
    return (res1 - res2)

def generate_perturbed_sicpovm_1qb_small(amt=0.1, delta=0.5):
    pauli = [
        np.array([[1,0],[0,1]], dtype=complex),
        np.array([[0,1],[1,0]], dtype=complex),
        np.array([[0,-1j],[1j,0]], dtype=complex),
        np.array([[1,0],[0,-1]], dtype=complex)
    ]
    n = random_direction()
    rotation = []
    for ni in n:
        rotation.append(0)
        for nj, sigma in zip(ni, pauli[1:]):
            rotation[-1] += sigma*nj
    basis_projs = [
        qi.Statevector([1,0]).to_operator(),
        qi.Statevector([1/np.sqrt(3),
                        np.sqrt(2/3)]).to_operator(),
        qi.Statevector([1/np.sqrt(3),
                        np.sqrt(2/3) * np.exp(1j*2*np.pi/3)]).to_operator(),
        qi.Statevector([1/np.sqrt(3),
                        np.sqrt(2/3) * np.exp(1j*4*np.pi/3)]).to_operator()
    ]
    new_povm = []
    for proj, rot in zip(basis_projs, rotation):
        sum_proj = np.zeros_like(proj.data)
        for operator in [proj2 for proj2 in basis_projs if proj2 is not proj]:
            sum_proj += commutator(operator.data, rot)
        new_proj = (proj.data/2) + 1j*np.pi*amt*(delta*commutator(proj.data, rot) - (1-delta)*sum_proj)
        new_povm.append(qi.Operator(new_proj))
    valid = True
    new_povm = Basis(new_povm, name=f'SIC-POVM (p={amt}, delta={delta})')
    valid = verify_povm(new_povm, print_res=True)
    return new_povm, valid

In [ ]:
def calculate_entropies(state):
    num = range(state.num_qubits)
    trace_out = generate_combinations(num)
    entropy = {}
    for subsys, cmplmt in zip(trace_out[:int(len(trace_out)/2)], 
                              trace_out[::-1]):
        state_traced = qi.partial_trace(state, subsys)
        entropy[f'{subsys} / {cmplmt}'] = qi.entropy(state_traced)
    return entropy


def solve_for_coefs(basis):
    dim = 2**basis[0].num_qubits
    idty = np.identity(dim).reshape(-1,1)
    matrix = []
    for i, operator in enumerate(basis):
        operator = operator.data.reshape(1,-1)[0]
        matrix.append(operator)
    matrix = np.array(matrix).T
    coefs = np.linalg.solve(matrix, idty).reshape(1,-1)[0]
    for i, coef in enumerate(coefs):
        basis[i] *= coef
    return Basis(basis)

In [2]:
rng = np.random.default_rng(seed = 12345)

# 1 qubit states
state_1qb = generate_n_qubits(1, 
                              entangled=True, 
                              density_matrix=True)

# 1 qubit mixed array
num_coefs = 100
coefs = np.linspace(0, 1, num_coefs)
var_states = []
for c in coefs:
    var_states.append(
        qi.DensityMatrix(
            [[(1-c)*state_1qb.data[0,0] + c/2, 
              (1-c)*state_1qb.data[0,1]],
             [(1-c)*state_1qb.data[1,0], 
              (1-c)*state_1qb.data[1,1] + c/2]]))

# 2 qubit states
state_2qb_sep = generate_n_qubits(2, 
                                  entangled=False, 
                                  density_matrix=True)
state_2qb_ent = generate_n_qubits(2, 
                                  entangled=True, 
                                  density_matrix=True)
state_bell = qi.DensityMatrix(
    qi.Statevector([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]))
state_mine = qi.DensityMatrix(
    qi.Statevector([1, 0]).tensor(
        qi.Statevector([1/np.sqrt(2), 1/np.sqrt(2)])))

# 3 qubit states
state_3qb_sep = generate_n_qubits(3, 
                                  entangled=False, 
                                  density_matrix=True)
state_3qb_ent = generate_n_qubits(3, 
                                  entangled=True, 
                                  density_matrix=True)
state_ghz = qi.DensityMatrix(
    qi.Statevector([1/np.sqrt(2),0,0,0,0,0,0,1/np.sqrt(2)]))
state_w = qi.DensityMatrix(
    qi.Statevector([0, 1/np.sqrt(3), 
                    1/np.sqrt(3), 0, 
                    1/np.sqrt(3), 0, 
                    0, 0]))

# 4 qubit states
state_4qb = generate_n_qubits(5, entangled=True)

In [ ]:
num_shots = 1000
step = 1
runs_per_shot = 1000
plt.figure(figsize=(14, 7))
for basis in [sicpovm_1qb, pauli_1qb]:
    data = qst(state_1qb, 
               basis, 
               num_shots, 
               runs_per_shot, 
               step)
    plt.plot(data.runs, 
             data.fidelity, 
             label=basis.name)
    plt.fill_between(data.runs, 
                     data.err_lo, 
                     data.err_hi, 
                     alpha=0.2)
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
num_samples = 1000
ptb_amts = np.linspace(0, 1, num_samples)
num_iter = 1000
p = []
norms2_valid = []
norms2_invalid = []
for i, amt in enumerate(ptb_amts):
    counts = 0
    norms1_valid = []
    norms1_invalid = []
    for _ in range(num_iter):
        povm, physical = generate_perturbed_sicpovm_1qb(amt)
        for proj, sicproj in zip(povm, sicpovm_1qb):
            norm = frobenius_norm(proj.data, sicproj.data)
        if physical:
            counts += 1
            norms1_valid.append(abs(norm)*4)
            norms1_invalid.append(np.nan)
        else:
            norms1_valid.append(np.nan)
            norms1_invalid.append(abs(norm)*4)
    p.append(counts / num_iter)
    norms2_valid.append(norms1_valid)
    norms2_invalid.append(norms1_invalid)

avg_valid = []
avg_invalid = []
for norms_valid, norms_invalid in zip(norms2_valid, 
                                      norms2_invalid):
    norms_valid = np.array(norms_valid)[~np.isnan(norms_valid)]
    norms_invalid = np.array(norms_invalid)[~np.isnan(norms_invalid)]
    avg_valid.append(np.mean(norms_valid))
    avg_invalid.append(np.mean(norms_invalid))

plt.figure(figsize=(18, 9))
seaborn.set_theme(style='whitegrid', 
                  palette='colorblind')
for i, norms in enumerate(np.array(norms2_invalid).T):
    if i == 0:
        label = 'Z invalid POVM'
    else:
        label = ''
    plt.scatter(ptb_amts, 
                norms, 
                color='black', 
                s=0.01, 
                label=label)
for i, norms in enumerate(np.array(norms2_valid).T):
    if i == 0:
        label = 'Z valid POVM'
    else:
        label = ''
    plt.scatter(ptb_amts, 
                norms, 
                color='green', 
                s=0.1, 
                label=label)
plt.plot(ptb_amts, 
         avg_invalid, 
         color='orangered', 
         linewidth=1, 
         label='Average Z invalid POVM')
plt.plot(ptb_amts, 
         avg_valid, 
         color='limegreen', 
         label='Average Z valid POVM')
plt.plot(ptb_amts, 
         p, 
         label='Probability')
plt.xlabel('Perturbation Amount')
plt.ylabel('Probability, Z')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

num_samples = 1000
ptb_amts = np.linspace(0, 0.001, num_samples)
num_iter = 1000
p = []
for i, amt in enumerate(ptb_amts):
    counts = 0
    for _ in range(num_iter):
        povm, physical = generate_perturbed_sicpovm_1qb(amt)
        if physical:
            counts += 1
    p.append(counts / num_iter)

plt.plot(ptb_amts, 
         p, 
         label='Probability')
plt.xlabel('Perturbation Amount')
plt.ylabel('Probability')
plt.grid(True)
#plt.legend(loc='lower right')
plt.show()

In [ ]:
rnd_povms = []
ptb_amts = []
num_samples = 10
num_iter = 10
for amt in np.linspace(0.5, 0.1, 5):
    povms = []
    valid_count = 0
    for _ in range(num_iter):
        valid = False
        i = 0
        while valid is False and i<1000:
            povm, valid = generate_perturbed_sicpovm_1qb(amt)
            i += 1
        if valid:
            povms.append(povm)
            valid_count += 1
    if valid_count > 0:
        rnd_povms.append(povms)
        ptb_amts.append([amt]*valid_count)
        
for amt in np.linspace(0.001, 0, 5):
    povms = []
    valid_count = 0
    for _ in range(num_iter):
        valid = False
        i = 0
        while valid is False and i<1000:
            povm, valid = generate_perturbed_sicpovm_1qb(amt)
            i += 1
        if valid:
            povms.append(povm)
            valid_count += 1
    if valid_count > 0:
        rnd_povms.append(povms)
        ptb_amts.append([amt]*valid_count)
        
num_shots = 200
step = 1
runs_per_shot = 1000
cmap = mpl.colormaps['viridis']
colors = cmap(np.linspace(0, 1, num_samples))
plt.figure(figsize=(14, 7))
for povm_rpt, ptb_rpt, color in zip(rnd_povms, 
                                    ptb_amts, 
                                    colors):
    data = []
    fidelity_rpt = []
    for povm, amt in zip(povm_rpt, ptb_rpt):
        data.append(
            qst(state_1qb, 
                povm, 
                num_shots, 
                runs_per_shot, 
                step,
                calc_errors=False))
        fidelity_rpt.append(data[-1].fidelity)
    plt.plot(data[0].runs, 
             np.mean(fidelity_rpt, axis=0), 
             color=color, 
             label=f'{ptb_rpt[0]:.4f}')
plt.xlabel('Number of Measurements')
plt.ylabel('Average State Fidelity')
plt.grid(True)
plt.legend(loc='lower right', 
           title='Perturbation:', 
           fontsize='x-small')
plt.show()

In [ ]:
num_shots = 400
step = 1
runs_per_shot = 1000
cmap = mpl.colormaps['viridis']
colors = cmap(np.linspace(0, 1, num_coefs))
plt.figure(figsize=(14, 7))
for state, color in zip(var_states, colors):
    data = qst(state, 
               sicpovm_1qb, 
               num_shots, 
               runs_per_shot, 
               calc_errors=False)
    purity = np.real(state.purity())
    plt.plot(data.runs, 
             data.fidelity, 
             color=color, 
             label=f'{purity:.2f}')
plt.xlabel('Number of Measurements')
plt.ylabel('Average State Fidelity')
plt.grid(True)
plt.legend(loc='lower right', 
           title='Purity:')
plt.show()

In [ ]:
purity = []
fidelities = []
errs_lo = []
errs_hi = []
errs_min = []
errs_max = []

num_shots = 4
step = 1
runs_per_shot = 1000
plt.figure(figsize=(14, 7))
for var_state in var_states:
    data = qst(
        var_state, 
        sicpovm_1qb, 
        num_shots, 
        runs_per_shot, 
        profile=False
    )
    fidelities.append(data.fidelity)
    errs_lo.append(data.err_lo)
    errs_hi.append(data.err_hi)
    errs_min.append(data.err_min)
    errs_max.append(data.err_max)
    purity.append(np.real(var_state.purity()))
plt.fill_between(purity, 
                 errs_lo, 
                 errs_hi, 
                 alpha=0.2)
plt.plot(purity, 
         errs_min, 
         color='black', 
         dashes=[3,2], 
         linewidth=1, 
         label='Min/Max Fidelity')
plt.plot(purity, 
         errs_max, 
         color='black', 
         dashes=[3,2], 
         linewidth=1)
plt.plot(purity, 
         fidelities, 
         label='Average Fidelity')
plt.xlabel('Purity')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
purity = []
fidelities = []
errs_lo = []
errs_hi = []
errs_min = []
errs_max = []

num_shots = 5000
step = 1
runs_per_shot = 500
for var_state in var_states:
    data = qst(
        var_state, 
        sicpovm_1qb, 
        num_shots, 
        runs_per_shot, 
        profile=False
    )
    fidelities.append(data.fidelity)
    errs_lo.append(data.err_lo)
    errs_hi.append(data.err_hi)
    errs_min.append(data.err_min)
    errs_max.append(data.err_max)
    purity.append(np.real(var_state.purity()))
plt.figure(figsize=(14, 7))
plt.fill_between(purity, 
                 errs_lo, 
                 errs_hi, 
                 alpha=0.2)
plt.plot(purity, 
         errs_min, 
         color='black', 
         dashes=[3,2], 
         linewidth=1, 
         label='Min/Max Fidelity')
plt.plot(purity, 
         errs_max, 
         color='black', 
         dashes=[3,2], 
         linewidth=1)
plt.plot(purity, 
         fidelities, 
         label='Average Fidelity')
plt.xlabel('Purity')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
num_shots = 10000
step = 1
runs_per_shot = 100
plt.figure(figsize=(14, 7))
labels_2qb = ['Random Separable State', 
          'Random Entangled State', 
          'Bell State', 
          'Separable State']
for state, label in zip([state_2qb_sep, 
                         state_2qb_ent, 
                         state_bell, 
                         state_mine], labels_2qb):
    data = qst(
        state, 
        sicpovm_2qb, 
        num_shots, 
        runs_per_shot, 
        step)
    plt.plot(data.runs, 
             data.fidelity, 
             label=label)
    plt.fill_between(data.runs, 
                     data.err_lo, 
                     data.err_hi, 
                     alpha=0.2)
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
num_shots = 10000
step = 1
runs_per_shot = 200
plt.figure(figsize=(14, 7))
for povm in [sicpovm_2qb, pauli_2qb]:
    data = qst(
        state_2qb_ent, 
        povm, 
        num_shots, 
        runs_per_shot, 
        step)
    plt.plot(data.runs, 
             data.fidelity, 
             label=povm.name)
    plt.fill_between(data.runs, 
                     data.err_lo, 
                     data.err_hi, 
                     alpha=0.2)
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
states_traced, trace_out = list_subsystems(state_3qb_sep)

num_shots = 200
step = 1
runs_per_shot = 1000
plt.figure(figsize=(14, 7))
for state, out in zip(states_traced, trace_out):
    runs, fidelity, err_min, err_max = qst(
        state, 
        sicpovm[state.num_qubits], 
        num_shots, 
        runs_per_shot, 
        step)
    entropy = qi.entropy(state)
    plt.plot(runs, 
             fidelity, 
             label=f'{out}, {entropy:.2f}')    
    plt.fill_between(runs, 
                     err_min, 
                     err_max, 
                     alpha=0.2)
plt.title('Fidelities of traced out Random Separable State')
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
states_traced, trace_out = list_subsystems(state_3qb_ent)

num_shots = 200
step = 1
runs_per_shot = 1000
plt.figure(figsize=(14, 7))
for state, out in zip(states_traced, trace_out):
    runs, fidelity, err_min, err_max = qst(
        state, 
        sicpovm[state.num_qubits], 
        num_shots, 
        runs_per_shot, 
        step)
    entropy = qi.entropy(state)
    plt.plot(runs, 
             fidelity, 
             label=f'{out}, {entropy:.2f}')    
    plt.fill_between(runs, 
                     err_min, 
                     err_max, 
                     alpha=0.2)
plt.title('Fidelities of traced out Random Entangled State')
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
states_traced, trace_out = list_subsystems(state_ghz)

num_shots = 200
step = 1
runs_per_shot = 1000
plt.figure(figsize=(14, 7))
for state, out in zip(states_traced, trace_out):
    data = qst(
        state, 
        sicpovm[state.num_qubits], 
        num_shots, 
        runs_per_shot, 
        step)
    entropy = qi.entropy(state)
    plt.plot(data.runs, 
             data.fidelity, 
             label=f'{out}, {entropy:.2f}')   
    plt.fill_between(data.runs, 
                     data.err_lo, 
                     data.err_hi, 
                     alpha=0.2)
plt.title('Fidelities of traced out GHZ State')
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
states_traced, trace_out = list_subsystems(state_w)

num_shots = 200
step = 1
runs_per_shot = 1000
plt.figure(figsize=(14, 7))
for state, out in zip(states_traced, trace_out):
    runs, fidelity, err_min, err_max = qst(
        state, 
        sicpovm[state.num_qubits], 
        num_shots, 
        runs_per_shot, 
        step)
    entropy = qi.entropy(state)
    plt.plot(runs, 
             fidelity, 
             label=f'{out}, {entropy:.2f}')
    plt.fill_between(runs, 
                     err_min, 
                     err_max, 
                     alpha=0.2)
plt.title('Fidelities of traced out W State')
plt.xlabel('Number of Measurements')
plt.ylabel('State Fidelity')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
#plt.figure(figsize=(20, 7))
fig = plt.figure(figsize=(12, 7), layout='tight')
gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
axs = gs.subplots(sharex=True, sharey=True)

num_shots = 200
step = 1
runs_per_shot = 5000

states_3qb = [state_3qb_sep,
              state_3qb_ent,
              state_ghz,
              state_w]
labels = ['A', 'B', 'C', 'D']
for ax, state_3qb, label in zip(axs.flatten(), states_3qb, labels):
    states_traced, trace_out = list_subsystems(state_3qb)
    for state, out in zip(states_traced, trace_out):
        data = qst(
            state, 
            sicpovm[state.num_qubits], 
            num_shots, 
            runs_per_shot, 
            step)
        entropy = qi.entropy(state)
        ax.plot(data.runs, 
                 data.fidelity, 
                 label=f'{out}, {entropy:.2f}')    
        ax.fill_between(data.runs, 
                         data.err_lo, 
                         data.err_hi, 
                         alpha=0.2)
    ax.text(200, 0.2, label, size=14,
         ha="center", va="center",
         bbox=dict(boxstyle="round",
                   fc=(0.9, 0.9, 0.9),
                   )
         )
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_color('black')
        ax.spines[axis].set_linewidth(0.5)
    #ax.set_xlim(0, num_shots)
    #ax.set_ylim(0, 1)
    #ax.xlabel('Number of Measurements')
    #ax.ylabel('State Fidelity')
    #ax.grid(True)
    #ax.legend(loc='lower right')

fig.supxlabel('Number of Measurements', size=10)
fig.supylabel('State Fidelity', size=10)
plt.show()

In [ ]:
def generate_pauli_matrices(num):
    pauli = [
        qi.Operator([[1,0],[0,1]]),
        qi.Operator([[0,1],[1,0]]),
        qi.Operator([[0,-1j],[1j,0]]),
        qi.Operator([[1,0],[0,-1]])
    ]
    temp_pauli = pauli
    new_pauli = pauli
    for _ in range(num-1):
        new_pauli = []
        for operator1 in temp_pauli:
            for operator2 in pauli:
                new_pauli.append(operator1.tensor(operator2))
        temp_pauli = new_pauli
    return new_pauli
    

def generate_even_combinations(lst):
    combinations = []
    r = len(lst)-1
    combinations.extend([list(comb) for comb in itertools.combinations(lst, r)])
    return combinations

num = 3
dim = 2**num
my_list = range(4**num-1)
combinations = generate_even_combinations(my_list)
vec = [1]*(4**num-1)
matrix = [vec]
for combo in combinations:
    newvec = np.array([1]*(4**num-1))
    for i in combo:
        newvec[i] = -1
    matrix.append(newvec)

matrix = np.array(matrix)
pauli_matrices = generate_pauli_matrices(num)
operators = []
for coefs in matrix:
    new_operators = qi.Operator(np.zeros((dim,dim), dtype=complex))
    for operator, coef in zip(pauli_matrices[1:], coefs):
        new_operators += qi.Operator(operator.data * coef)/dim
    operators.append(new_operators)
new_sic = []
for operator in operators:
    new_sic.append((1/dim)*(pauli_matrices[0] + (1/np.sqrt(4**num-1))*operator))
new_sic = Basis(new_sic)
atl(new_sic.data[1])
for operator1 in new_sic:
    for operator2 in new_sic:
        print((operator1 @ operator2).data.trace())